In [ ]:
from skimage import segmentation, io
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import calinski_harabasz_score
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
import pandas as pd
from utils.utils import generate_synthetic_image, plot_dendrogram
from utils.image_manager import ImagesManager
from utils.mask_manager import MaskManager
from utils.features_manager import FeaturesManager
import os
import warnings
plt.rcParams["axes.grid"] = False
from sklearn.preprocessing import scale

In [ ]:
model = keras.models.load_model('./model/model.h5')

In [ ]:
#path_to_images = './data/eval_xai_method/dog_resized'
path_to_images ='./data/debug'
all_images = os.listdir(path_to_images)
all_images = [x for x in all_images if x != '.DS_Store']

In [ ]:
all_images

In [ ]:
num_images = len(all_images)

In [ ]:
images_chosen = np.random.choice(all_images, size=num_images, replace=False)

In [ ]:
images_chosen

In [ ]:
prediction_list = []
for image_name in images_chosen:
    image_path = os.path.join(path_to_images, image_name)
    img = io.imread(image_path)
    img_norm = img/255
    img_batch = np.expand_dims(img_norm, axis=0)
    pred = model.predict(img_batch)
    prediction_list.append(pred)
    io.imshow(img)
    plt.show()
    print(pred)

In [ ]:
ratio_pixels_gaussian_kernel = 0.0075 #Corresponde a sigma=4 para una imagen 200x200
chi2_95 = 6

for image_name in images_chosen[:1]:
    image_path = os.path.join(path_to_images, image_name)
    img = io.imread(image_path)
    img_norm = img/255
    x_pixels, y_pixels, _ = img_norm.shape
    area_total = x_pixels*y_pixels
    s = np.sqrt(area_total)/200
    max_dist = 10*s
    ratio = 1/2*s
    sigma_sq = area_total*ratio_pixels_gaussian_kernel/(chi2_95*np.pi)
    sigma = np.sqrt(sigma_sq)
    superpixels = segmentation.quickshift(img_norm, kernel_size=sigma, max_dist=max_dist, ratio=ratio)
    num_superpixels_image = np.unique(superpixels).shape[0]
    img_batch = np.expand_dims(img_norm, axis=0)
    pred = model.predict(img_batch)
    print('num_superpixels:', num_superpixels_image)
    print('image_shape:', img_norm.shape)
    print('sigma:', sigma)
    print('max_dist:', max_dist)
    print('ratio:', ratio)
    io.imshow(segmentation.mark_boundaries(img_norm, superpixels, color=(1,0,0), mode='inner'))
    plt.show()

In [ ]:
import os
from skimage import segmentation, io
import cv2

class SiftFeaturesManager:
    def __init__(self, location, image_name, mask):
        self.location = location
        self.image_name = image_name
        self.mask = mask
        self.list_descriptors = []
        self.list_tracking = []
        self.list_missing = []
        
    def get_smalles_patch(self, img, mask, num_superpixel):
        
        idx_mask = np.where(mask == num_superpixel)
        min_idx = np.min(idx_mask, axis=-1)
        max_idx = np.max(idx_mask, axis=-1)
        slicer = []
        for min_val, max_val in zip(min_idx, max_idx):
            slice_dim = slice(min_val, max_val+1, 1)
            slicer.append(slice_dim)
            
        slicer = tuple(slicer)
        sliced_image = img[slicer]
        return img[slicer]
    
    def sift_features(self, s_patch):
        sift = cv2.SIFT_create(nfeatures=1)
        _, descriptors = sift.detectAndCompute(s_patch, None)
        return descriptors
    
    def prepare_output(self, descriptors, num_superpixel):
        if descriptors is not None:
            total_descriptors = descriptors.shape[0]
            total_features = descriptors.shape[1]
            for i in range(total_descriptors):
                des = descriptors[i, :]
                des = des.reshape((1, total_features))
                self.list_descriptors.append(des)
                self.list_tracking.append((self.image_name, num_superpixel))
        else:
            self.list_missing.append((self.image_name, num_superpixel))
            
    def create_output(self):
        if len(self.list_descriptors)>0:
            data_np = np.concatenate(self.list_descriptors)
            data_pd = pd.DataFrame(data_np)
            data_pd.index = self.list_tracking
        else:
            data_pd = None
        
        return data_pd, self.list_missing
        
    def get_sift_features(self):
        # Read the image
        full_path = os.path.join(self.location, self.image_name)
        img = io.imread(full_path)
        
        # For each superpixel:
        list_superpixels = np.unique(mask)
        for num_superpixel in list_superpixels:
            # Get the smalles patch that contains the superpixel
            smallest_patch = self.get_smalles_patch(img, mask, num_superpixel)
            # Get SIFT features
            descriptors = self.sift_features(s_patch=smallest_patch)
            self.prepare_output(descriptors, num_superpixel)
        
        data, list_missing = self.create_output()
        return data, list_missing

In [ ]:
def get_superpixels_information(path_to_images, images_extension='jpg',
                                kernel_size=4, max_dist=10, ratio=.3):
    
    images_manager = ImagesManager(location=path_to_images, images_extension=images_extension)
    images_names = images_manager.get_images_names()
    df_features_list = []
    dict_masks = {}
    dict_deleted_superpixels = {}
    num_superpixels_list = []

    for image_name in images_names:
        print(image_name)
        masks_manager = MaskManager(location=path_to_images, image_name=image_name)
        mask = masks_manager.get_mask(kernel_size=kernel_size, max_dist=max_dist, ratio=ratio)
        dict_masks[image_name] = mask
        num_superpixels_list.append(len(np.unique(mask)))
        sift_features_manager = SiftFeaturesManager(location=path_to_images, image_name=image_name, mask=mask)
        data, list_missing = sift_features_manager.get_sift_features()
        
    return data, list_missing, num_superpixels_list

In [ ]:
data, list_missing, num_superpixels_list = get_superpixels_information(path_to_images='./data/debug')

In [ ]:
data.head()

In [ ]:
class Clusteriser:
    
    def __init__(self, num_superpixels, df):
        self.num_superpixels = num_superpixels
        self.image_name = image_name
        self.df = df

    def get_fixed_clusters(self, n_cluster):
        
        cluster = AgglomerativeClustering(n_clusters=n_cluster, linkage='ward')
        results = cluster.fit(self.df)
        labels = results.labels_
        metric = calinski_harabasz_score(self.df, labels)

        return metric, labels
    
    def get_mutiple_clusters(self):

        mean_superpixels = round(np.mean(self.num_superpixels))
        n_clusters = range(2, mean_superpixels)
        n_clusters_list = []
        metric_list = []

        for n_cluster in n_clusters:
            metric, _ = self.get_fixed_clusters(n_cluster)
            metric_list.append(metric)
            n_clusters_list.append(n_cluster)
            print('n_cluster:', n_cluster, 'metric:', metric)

        return n_clusters_list, metric_list
        
    def get_best_clusterisation(self):
        n_clusters_list, metric_list = self.get_mutiple_clusters()
        idx_best_metric = np.argmax(metric_list)
        best_n_clusters = n_clusters_list[idx_best_metric]

        _, labels = self.get_fixed_clusters(best_n_clusters)
        return np.array(labels)

In [ ]:
cluster = Clusteriser(num_superpixels=num_superpixels_list, df=data)
new_segmentation = cluster.get_best_clusterisation()

# Trash